# TMA4212: Project 2

#### Remark: Run the code in the given order. If memmory or error problems occur, clear the kurnel.  

## Task 1a-c: with a regular grid on $[0,1]^2$

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import time
from matplotlib import cm



In [2]:
# Setting parameters
a = 0.1
r = 1

### Defining the finite difference matrix

In [3]:
def matrix_A(M):

        # Main tridiag
        diagonal = 2*(1+a) * np.ones((M-1)**2)
        off_diag = -a * np.ones((M-1)**2-1)
        off_diag[(M-2)::(M-1)] = np.zeros(M-2)

        # Far off-diag
        far_off_diag = -np.ones((M-2)*(M-1)-1)
        far_off_diag[(M-2)::(M-1)] = np.zeros(M-3)

        return np.diag(diagonal)+np.diag(off_diag,1)+np.diag(off_diag,-1)+np.diag(far_off_diag,M)+np.diag(far_off_diag,-M)


### Constructing the RHS with BCs

In [4]:
def RHS_maker(BC,RHS,M):
        h = 1/M

        # BC, up/down and sides
        BC_ud = np.zeros((M-1)**2)
        BC_ud[1:(M-1)] = BC[0,1:-2]
        BC_ud[(M-1)*(M-2):-1] = BC[-1,2:-1]

        BC_sides = np.zeros((M-1)**2)
        BC_sides[:(M-1)*(M-2)+1:(M-1)] = a*BC[1:-1,0]+BC[0:-2,0]
        BC_sides[(M-2)::(M-1)] = a*BC[1:-1,-1] + BC[2:,-1]

        # RHS, se mer på rekkefølge, men ikke viktig nå...
        RHS_here = np.reshape(RHS[1:-1,1:-1],-1)
        return h**2*RHS_here + BC_sides + BC_ud

### Defining the scheme

In [5]:
def Regular1(BC,RHS,M):
        # Solve Fourier's law for the heat flux
        # in anisotropic material with r = 1.
        # Input:
        #       BC: boundary conditions for domain [0,1]^2,
        #           random entries for the rest...
        #       RHS: RHS of equation...
        #       M: number of points in each direction

        U = BC

        h = 1/M
        A = matrix_A(M)

        RHS_w_BC = RHS_maker(BC,RHS,M)

        U_vec = np.linalg.solve(A,RHS_w_BC)
        U[1:-1,1:-1] = np.reshape(U_vec,(-1,M-1))

        print(h,M)

        return U

### Defining the convergence function

In [6]:
def convergence(u_exact, u_exact_BC, u_exact_RHS, solver=Regular1):
        P = 4
        Hconv = np.zeros(P)
        Econv = np.zeros(P)
        M = 10
        for p in range(P):
                BC = u_exact_BC(u_exact,M)
                RHS = u_exact_RHS(u_exact,M)
                U = solver(BC, RHS, M=M)
                Hconv[p] = 1/M
                X,Y,u = u_exact(M)
                Econv[p] = np.max(np.abs(u-U))
                M = 2*M
        order = np.polyfit(np.log(Hconv),np.log(Econv),1)[0]
        return Hconv, Econv, order

### Example 1

$u = y\sin(\pi x)$

$f = \pi^2 y(1+a)\sin(\pi x)-2r\pi\cos(\pi x)$

In [7]:
def u_ex(M):
        x = np.linspace(0,1,M+1)
        y = np.linspace(0,1,M+1)
        X,Y = np.meshgrid(x,y)
        return X, Y, Y*np.sin(np.pi*X)
def u_ex_BC(u_ex,M):
        X,Y,BC = u_ex(M)
        BC[1:-1,1:-1] = np.random.rand(M-1,M-1)
        return BC
def u_ex_RHS(u_ex,M):
        X,Y,u = u_ex(M)
        RHS = np.pi**2*Y*(1+a)*np.sin(np.pi*X)-2*r*np.pi*np.cos(np.pi*X)
        return RHS

### Example 2

$u = y^2 \arctan(x)$

$f = -\frac{2}{x^2+1}(2y-\frac{(1+a)xy^2)}{x^2+1}) - 2\arctan(x)$

In [8]:
def u_ex_2(M):
        x = np.linspace(0,1,M+1)
        y = np.linspace(0,1,M+1)
        X,Y = np.meshgrid(x,y)
        return X, Y, Y*Y*np.arctan(X)
def u_ex_BC_2(u_ex,M):
        X,Y,BC = u_ex(M)
        BC[1:-1,1:-1] = np.random.rand(M-1,M-1)
        return BC
def u_ex_RHS_2(u_ex,M):
        X,Y,u = u_ex(M)
        onesM = np.ones((M+1,M+1))
        RHS = -2/(X*X+onesM)*(2*Y-(1+a)*X*Y*Y/(X*X+onesM)) - 2*np.arctan(X)
        return RHS

### Testing the scheme for the examples, with convergence plot

In [ ]:
# Grid
M = 100

# Example 1
X,Y, u = u_ex(M)
BC = u_ex_BC(u_ex,M)
RHS = u_ex_RHS(u_ex,M)
U1 = Regular1(BC,RHS,M)

# Example 2
X,Y, u2 = u_ex_2(M)
BC2 = u_ex_BC_2(u_ex_2,M)
RHS2 = u_ex_RHS_2(u_ex_2,M)
U2 = Regular1(BC2,RHS2,M)

# Figure
fig = plt.figure()

# Plotting solution for example 1
ax1 = fig.add_subplot(2,2,1, projection='3d')
ax1.plot_surface(X,Y,U1)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('U')

# Plotting solution for example 2
ax2 = fig.add_subplot(2,2,2, projection='3d')
ax2.plot_surface(X,Y,U2)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('U')

Hconv1, Econv1, order1 = convergence(u_ex,u_ex_BC,u_ex_RHS,Regular1)
Hconv2, Econv2, order2 = convergence(u_ex_2,u_ex_BC_2,u_ex_RHS_2,Regular1)

# Plotting error convergence for example 1
ax3 = fig.add_subplot(2,2,3)
ax3.loglog(Hconv1,Econv1,'o-', label='p={:.2f}'.format(order1))
ax3.set_xlabel('h')
ax3.set_ylabel('error')
ax3.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax3.legend()

# Plotting error convergence for example 2
ax4 = fig.add_subplot(2,2,4)
ax4.loglog(Hconv2,Econv2,'o-', label='p={:.2f}'.format(order2))
ax4.set_xlabel('h')
ax4.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax4.legend()

plt.show()

## Task 1d: with an irregular grid on $[0,1]^2$

In [ ]:
# Renewing parameters
a = 0.1 # same as before
r = np.sqrt(2)

### Defining finite difference matrix

In [ ]:
def matrix_A_irregular(M):
        N = np.ceil(M/r).astype('int')

        # Main tridiag
        diagonal = 2*(1+a) * np.ones((M-1)*(N-1))
        off_diag = -a * np.ones((M-1)*(N-1)-1)
        off_diag[(M-2)::(M-1)] = np.zeros(N-2)

        # Far off-diag
        far_off_diag = -np.ones((M-1)*(N-2)-1)
        far_off_diag[(M-2)::(M-1)] = np.zeros(N-3)

        return np.diag(diagonal)+np.diag(off_diag,1)+np.diag(off_diag,-1)+np.diag(far_off_diag,M)+np.diag(far_off_diag,-M)

### Defining RHS-maker with BCs for the irregular grid

In [ ]:
def RHS_maker_irregular(BC,RHS,M,upper_BC):
        h = 1/M
        k = r*h
        N = np.ceil(M/r).astype('int')
        eta = (k-N*k+1)/(r*h)

        x = np.linspace(eta*h,1+eta*h,M+1)

        print('N','eta, q')
        print(N, eta, k-(N*k-1))

        # BC, up/down and sides
        BC_ud = np.zeros((M-1)*(N-1))
        BC_ud[1:(M-1)] = BC[0,1:-2]
        BC_ud[(M-1)*(N-2):] = upper_BC(x[1:-1])

        BC_sides_direc = np.zeros((M-1)*(N-1))
        BC_sides_direc[(M-2):(M-1)*(N-1)+1:(M-1)] = BC[2:,-1]
        BC_sides_direc[:(M-1)*(N-2)+1:(M-1)] = BC[:-2,0]

        BC_sides = np.zeros((M-1)*(N-1))
        BC_sides[:(M-1)*(N-2)+1:(M-1)] = a*BC[1:-1,0]
        BC_sides[(N-2)::(M-1)] = a*BC[1:-1,-1]

        # RHS
        RHS_here = np.reshape(RHS[1:-1,1:-1],-1)

        return h**2*RHS_here + BC_sides + BC_ud + BC_sides_direc

### Defining the schemes

In [ ]:
def Irregular1(BC,RHS,M, upper_BC):
        # Solve Fourier's law for the heat flux
        # in anisotropic material with r = sqrt(2).
        # Input:
        #       BC: boundary conditions for domain [0,1]^2,
        #           random entries for the rest...
        #       RHS: RHS of equation...
        #       M: number of points in x-direction
        #       upper_BC: function which has BC on upper boundary as
        #               a function of x

        U = BC

        h = 1/M
        N = np.ceil(M/r).astype('int')

        A = matrix_A_irregular(M)
        RHS_w_BC = RHS_maker_irregular(BC,RHS,M,upper_BC)

        U_vec = np.linalg.solve(A,RHS_w_BC)
        U[1:-1,1:-1] = np.reshape(U_vec,(-1,M-1))
        return U

### Convergence function

In [ ]:
def convergence_irregular(u_exact, u_exact_BC, u_exact_RHS, upper_BC,solver=Irregular1):
        P = 4
        Hconv = np.zeros(P)
        Econv = np.zeros(P)
        M = 10
        for p in range(P):
                BC = u_exact_BC(u_exact,M)
                RHS = u_exact_RHS(u_exact,M)
                U = solver(BC, RHS, M,upper_BC)
                Hconv[p] = 1/M
                X,Y,u = u_exact(M)
                Econv[p] = np.max(np.abs(u-U))
                M = 2*M
        order = np.polyfit(np.log(Hconv),np.log(Econv),1)[0]
        return Hconv, Econv, order

### Example 3

$u = y\sin(\pi x)$

$f = \pi^2 y(1+a)\sin(\pi x)-2r\pi\cos(\pi x)$

In [ ]:

# Example 3?
def u_ex(M):
        N = np.ceil(M/r).astype('int')
        x = np.linspace(0,1,M+1)
        y = np.ones(N+1)
        y[:-1] = np.linspace(0,r*(N-1)/M,N)
        X,Y = np.meshgrid(x,y,indexing='xy')
        return X, Y, Y*np.sin(np.pi*X)
def u_ex_BC(u_ex,M):
        N = np.ceil(M/r).astype('int')
        X,Y,BC = u_ex(M)
        BC[1:-1,1:-1] = np.zeros((N-1,M-1))
        return BC
def u_ex_RHS(u_ex,M):
        X,Y,u = u_ex(M)
        RHS = np.pi**2*Y*(1+a)*np.sin(np.pi*X)-2*r*np.pi*np.cos(np.pi*X)
        return RHS
def upper_BC(x):
        return np.sin(np.pi*x)

### Testing the scheme for example 3 with convergence plot

In [ ]:
# Solution to example 3
X,Y, u = u_ex(M)
BC = u_ex_BC(u_ex,M)
RHS = u_ex_RHS(u_ex,M)

U1 = Irregular1(BC,RHS,M,upper_BC)

# Figure
fig = plt.figure(figsize = [4.8, 2.4])

# Plot of solution
ax1 = fig.add_subplot(1,2,1, projection='3d')
ax1.plot_surface(X,Y,U1)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('U=U(x,y)')

Hconv1, Econv1, order1 = convergence_irregular(u_ex,u_ex_BC,u_ex_RHS,upper_BC,Irregular1)

# Convergence plot
ax2 = fig.add_subplot(1,2,2)
ax2.loglog(Hconv1,Econv1,'o-', label='p={:.2f}'.format(order1))
ax2.set_xlabel('h')
ax2.set_title('Error')
ax2.legend()
ax2.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

plt.tight_layout()
plt.show()

## Task 3a and 3c: A variable coefficient transport equation 

### Test functions

In [ ]:
T=0.1 # Time domain variable 

def a(x): # Sign changing on [0,1]
    return  0.5-np.exp(-5*(x-0.5)**2)

def ax(x): # Derivative of a w.r.t x 
    return 10*(x-1/2)*np.exp(-5*(x-1/2)**2)

def u0(x): # u(x,0)=u0(x)
    return 0
    
def g1(t): # Boundary condition on the inflow boundary 
    return np.sin(t)
def g2(t):
    return t

def f1(x,t): # RHS
    return np.cos(t)*np.cos(2*np.pi*x) - a(x)*2*np.pi*np.sin(t)*np.sin(2*np.pi*x)
def f2(x,t):
    return (1-t*a(x))*np.exp(-x)

def r(x,M,N): # a*(k/h), simplifies notation in the scheme
    h=1/M
    k=T/N
    return a(x)*k/h

def rx(x,M,N): # Derivative of r w.r.t x
    h=1/M
    k=T/N
    return ax(x)*k/h

def uExact1(x,t): # Exact test solution w.r.t our RHS
    return np.cos(2*np.pi*x)*np.sin(t)
def uExact2(x,t):
    return t*np.exp(-x)

def ft1(x,t): # Derivative of our RHS w.r.t t
    return -np.sin(t)*np.cos(2*np.pi*x) - a(x)*np.sin(t)*np.cos(2*np.pi*x)
def ft2(x,t):
    return -a(x)*np.exp(-x)

def fx1(x,t): # Derivative of our RHS w.r.t x
    return -2*np.pi*np.sin(2*np.pi*x)*np.cos(t) + ax(x)*np.cos(t)*np.cos(2*np.pi*x) - 2*np.pi*a(x)*np.sin(2*np.pi*x)*np.cos(t)
def fx2(x,t):
    return np.exp(-x)*(-1-t*ax(x)+t*a(x))

def RHS(x,t,N,f,ft,fx): # RHS for the Lax-Wendroff scheme
    k = T/N
    return k*f(x,t) + (1/2)*k**2*ft(x,t) - (1/2)*k**2*a(x)*fx(x,t)

### Defining the finite difference matrix for both schemes

In [ ]:
def matrixLax(M,N, xGrid): # Returns a M-1xM-1 matrix for the Lax-Wendroff main loop
    rPluss = np.zeros(M-1)
    rMinus = np.zeros(M-1)
    rDiag = np.zeros(M-1)
    
    h=1/M
    
    rVal = r(xGrid[1:-1],M,N)
    rxVal = rx(xGrid[1:-1],M,N)
    
    rPluss = (1/2)*rVal**2 - (1/2)*rVal + (1/4)*h*rVal*rxVal
    rMinus = (1/2)*rVal**2 + (1/2)*rVal - (1/4)*h*rVal*rxVal
    rDiag = np.ones(M-1) - rVal**2

    rMinus = np.delete(rMinus, 0) 
    rPluss = np.delete(rPluss, -1) 
    return np.diag(rMinus,-1)+np.diag(rDiag)+np.diag(rPluss,1)

In [ ]:
def matrixUpwind(M,N, xGrid): # Returns a MxM matrix for the upwind main loop
    print(M,N)
    rVal = np.zeros((M,2))
    
    rVal[:,0] = a(xGrid[1:])*M*T/N
    rPluss = rVal.max(1)
    rMinus = (-rVal).max(1)
              
    rDiag = np.ones(M) - rPluss - rMinus
    
    rMinus = np.delete(rMinus, -1)
    rPluss = np.delete(rPluss, 0)
    
    return np.diag(rPluss,-1)+np.diag(rDiag)+np.diag(rMinus,1)


### Defining the schemes

In [ ]:
def laxW(g,u0,f,ft,fx,M,N):
    # Solves the transport equation
    # u_t+a(x)=0, 0<=x<=1
    # with boundary conditions u(0,t)=g(t)
    # and initial values u(x,0)=u0(x)
    # over the time interval from 0 to T.
    # Input: 
    #       M, N: number of grid intervals in the x- and t directions
    #          g: function at x=0
    #         u0: function at t=0
    #          f: RHS
    #         ft: t derivative of RHS
    #         fx: x derivative of RHS
    # Output: 
    #       x, t: the gridpoints in the x- and t- directions 
    #       U: An array with the numerical solution.
    
    # Set the gridpoints
    tGrid = np.linspace(0,T,N+1) # Gridpoints in time
    xGrid = np.linspace(0,1,M+1) # Gridpoints in space
    
    # Set the stepsizes
    h = 1/M     # Stepsize in space
    k = T/N     # Stepsize in time
    
    A = matrixLax(M,N, xGrid) # M-1xM-1 matrix for the main loop
    U = np.zeros((N+1,M+1))   # Storage unit for our solution
    fn = np.zeros(M-1)        # Storage unit for the RHS and the boundary 
    
    U[:,0] = g(tGrid)         # Set U_0 for every t_n
    U[0,:] = u0(xGrid[:])     # Set U^0 for every x_m
    
    r1 = r(xGrid[1],M,N)      # r_1
    rM = r(xGrid[-2],M,N)     # r_m-1
    
    for j in range(N):
        fn[:] = RHS(xGrid[1:-1],tGrid[j],N,f,ft,fx)
        
        fn[0] += g(tGrid[j])*((1/2)*r1**2 + (1/2)*r1 - (1/4)*h*r1*rx(xGrid[1],M,N)) #Boundry condition at x=0
        fn[-1] += U[j,-1]*((1/2)*rM**2 - (1/2)*rM + (1/4)*h*rM*rx(xGrid[1],M,N))    #Constant extention from U_M-1 condtition
 
        U[j+1,1:-1] = A@U[j,1:-1] + fn
        U[j+1,-1] = U[j+1,-2]  # Constant extension to the boundary 
    return xGrid, tGrid, U

In [ ]:
def upwindScheme(g,u0,f,M,N):
    # Solves the transport equation
    # u_t+a(x)=0, 0<=x<=1
    # with boundary conditions u(0,t)=g(t)
    # and initial values u(x,0)=u0(x)
    # over the time interval from 0 to T.
    # Input: 
    #       M, N: number of grid intervals in the x- and t directions
    # Output: 
    #       x, t: the gridpoints in the x- and t- directions 
    #       U: An array with the numerical solution.

    # Set the gridpoints
    tGrid = np.linspace(0,T,N+1)  # Gridpoints in time
    xGrid = np.linspace(0,1,M+1)  # Gridpoints in space
     
    A = matrixUpwind(M,N, xGrid)  # Matrix for the main loop
    fn = np.zeros(M)              # Storage unit for the RHS and the boundary 
    U = np.zeros((N+1,M+1))       # Storage unit for our solution 
    
    U[:,0] = g(tGrid)  # Set U_0 for all t_n
    U[0,:] = u0(xGrid) # Set U^0 for all x_m
     
    # Main loop
    for j in range(N):
        fn[:] = f(xGrid[1:],tGrid[j])*(T/N)         # Set the RHS
        fn[0] += g(tGrid[j])*max(r(xGrid[1],M,N),0) # Set the inflow condition 
        
        U[j+1,1:] = A@U[j,1:] + fn                  # iterate 
    return xGrid, tGrid, U


### Defining the convergence functions

In [ ]:
def convergenceLax(uExact,g,u0,f,ft,fx): 
    P = 4
    Hconv = np.zeros(P)
    Econv = np.zeros(P)
    M = 10
    for p in range(P):      
        x, t, U = laxW(g,u0,f,ft,fx,M=M, N=1000)
        U = np.transpose(U)
        Eh = uExact(x,t[-1])-U[:,-1]
        Hconv[p] = (x[1]-x[0])
        Econv[p] = np.max(np.abs(Eh))
        M = 2*M
    order = np.polyfit(np.log(Hconv),np.log(Econv),1)[0]
    return Hconv, Econv, order

In [ ]:
def convergence1Lax(U_ref,uExact,g,u0,f,ft,fx): 
    P = 4
    Tconv = np.zeros(P)
    Econv = np.zeros(P)
    N = 400
    
    for p in range(P):
        x, t, U = laxW(g,u0,f,ft,fx,M=20, N=N)
        U = np.transpose(U)
        
        Eh = U_ref[:,-1]-U[:,-1]
        Tconv[p] = (t[1]-t[0])
        Econv[p] = np.max(np.abs(Eh))
        N = 2*N
    order = np.polyfit(np.log(Tconv),np.log(Econv),1)[0]
    return Tconv, Econv, order

In [ ]:
def convergenceUp(uExact,g,u0,f):
    P = 4
    Hconv = np.zeros(P)
    Econv = np.zeros(P)
    M = 10
    for p in range(P):      
        x, t, U = upwindScheme(g,u0,f,M=M, N=1000)
        U = np.transpose(U)
        Eh = uExact(x,t[-1])-U[:,-1]
        Hconv[p] = (x[1]-x[0])
        Econv[p] = np.max(np.abs(Eh))
        M = 2*M
    order = np.polyfit(np.log(Hconv),np.log(Econv),1)[0]
    return Hconv, Econv, order

In [ ]:
def convergence1Up(U_ref,uExact,g,u0,f): 
    P = 4
    Tconv = np.zeros(P)
    Econv = np.zeros(P)
    N = 400
    
    for p in range(P):
        x, t, U = upwindScheme(g,u0,f,M=20, N=N)
        U = np.transpose(U)
        
        Eh = U_ref[:,-1]-U[:,-1]
        Tconv[p] = (t[1]-t[0])
        Econv[p] = np.max(np.abs(Eh))
        N = 2*N
    order = np.polyfit(np.log(Tconv),np.log(Econv),1)[0]
    return Tconv, Econv, order

### Testing the upwind scheme for $u_1$ and $u_2$ with convergence plots

In [ ]:
x5, t5, U_ref3 = upwindScheme(g1,u0,f1,M=20, N=10000)
U_ref3 = np.transpose(U_ref3)
x6, t6, U_ref4 = upwindScheme(g2,u0,f2,M=20, N=10000)
U_ref4 = np.transpose(U_ref4)

In [ ]:
fig4 = plt.figure(figsize=(13,13))
ax5 = fig4.add_subplot(3,2,1)
ax5.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

# Solve the equation
upx1, upt1, upU1 = upwindScheme(g1,u0,f1,20, 400)
upU1 = np.transpose(upU1)
# Plot the solution at some points in time
#ax5.figure(1)
#ax5.clf()
tplots = np.linspace(0,upt1[-1],6)
k = upt1[1]-upt1[0]
for tn in tplots:
    n = int(tn/k)
    tn = n*k
    ax5.plot(upx1,upU1[:,n])#,label='t={:.1f}'.format(tn))
ax5.set_xlabel('x')
ax5.set_ylabel('u1(x,t)')
ax5.legend()
#ax5.show()

ax6 = fig4.add_subplot(3,2,2)
ax6.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

# Solve the equation
upx2, upt2, upU2 = upwindScheme(g2,u0,f2,M=20, N=400)
upU2 = np.transpose(upU2)
# Plot the solution at some points in time
#ax6.figure(1)
#ax6.clf()
tplots = np.linspace(0,upt2[-1],6)
k = upt2[1]-upt2[0]
for tn in tplots:
    n = int(tn/k)
    tn = n*k
    ax6.plot(upx2,upU2[:,n],label='t={:.2f}'.format(tn))
ax6.set_xlabel('x')
ax6.set_ylabel('u2(x,t)')
ax6.legend(loc=1, prop={'size': 10})


H3, E3, p3 = convergenceUp(uExact1,g1,u0,f1)

ax1 = fig4.add_subplot(3,2,3)
ax1.loglog(H3,E3,'o-', label='p={:.2f}'.format(p3))
ax1.grid('on')
ax1.set_xlabel('h')
ax1.set_ylabel('error in space')
ax1.legend();

ax1.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

H4, E4, p4 = convergenceUp(uExact2,g2,u0,f2)

ax2 = fig4.add_subplot(3,2,4)
ax2.loglog(H4,E4,'o-', label='p={:.2f}'.format(p4))
ax2.grid('on')
ax2.set_xlabel('h')
ax2.legend();

ax2.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

#plt.tight_layout()



ax3 = fig4.add_subplot(3,2,5)


T5, E5, p5 = convergence1Up(U_ref3,uExact1,g1,u0,f1)
ax3.loglog(T5,E5,'o-', label='p={:.2f}'.format(p5))
ax3.grid('on')
ax3.set_xlabel('t')
ax3.set_ylabel('error in time')
ax3.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax3.legend();


ax4 = fig4.add_subplot(3,2,6)


T6, E6, p6 = convergence1Up(U_ref4,uExact2,g2,u0,f2)
ax4.loglog(T6,E6,'o-', label='p={:.2f}'.format(p6))
ax4.grid('on')
ax4.set_xlabel('t')

ax4.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax4.legend();
plt.savefig("UpPlot.pdf")



### Testing the Lax-Wendroff scheme for $u_1$ and $u_2$ with convergence plots

In [ ]:
x3, t3, U_ref1 = laxW(g1,u0,f1,ft1,fx1,M=20, N=10000)
U_ref1 = np.transpose(U_ref1)
x4, t4, U_ref2 = laxW(g2,u0,f2,ft2,fx2,M=20, N=10000)
U_ref2 = np.transpose(U_ref2)

In [ ]:
fig3 = plt.figure(figsize=(13,13))
ax5 = fig3.add_subplot(3,2,1)
ax5.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

# Solve the equation
laxx1, laxt1, laxU1 = laxW(g1,u0,f1,ft1,ft1, M=20, N=400)
laxU1 = np.transpose(laxU1)
# Plot the solution at some points in time
#ax5.figure(1)
#ax5.clf()
tplots = np.linspace(0,laxt1[-1],6)
k = laxt1[1]-laxt1[0]
for tn in tplots:
    n = int(tn/k)
    tn = n*k
    ax5.plot(laxx1,laxU1[:,n])#,label='t={:.1f}'.format(tn))
ax5.set_xlabel('x')
ax5.set_ylabel('u1(x,t)')
ax5.legend()
#ax5.show()

ax6 = fig3.add_subplot(3,2,2)
ax6.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

# Solve the equation
laxx2, laxt2, laxU2 = laxW(g2,u0,f2,ft2,ft2, M=20, N=400)
laxU2 = np.transpose(laxU2)
# Plot the solution at some points in time
#ax6.figure(1)
#ax6.clf()
tplots = np.linspace(0,laxt2[-1],6)
k = laxt2[1]-laxt2[0]
for tn in tplots:
    n = int(tn/k)
    tn = n*k
    ax6.plot(laxx2,laxU2[:,n],label='t={:.2f}'.format(tn))
ax6.set_xlabel('x')
ax6.set_ylabel('u2(x,t)')
ax6.legend(loc=1, prop={'size': 10})



H1, E1, p1 = convergenceLax(uExact1,g1,u0,f1,ft1,fx1)

ax1 = fig3.add_subplot(3,2,3)
ax1.loglog(H1,E1,'o-', label='p={:.2f}'.format(p1))
ax1.grid('on')
ax1.set_xlabel('h')
ax1.set_ylabel('error in space')
ax1.legend();

ax1.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

H2, E2, p2 = convergenceLax(uExact2,g2,u0,f2,ft2,fx2)

ax2 = fig3.add_subplot(3,2,4)
ax2.loglog(H2,E2,'o-', label='p={:.2f}'.format(p2))
ax2.grid('on')
ax2.set_xlabel('h')
ax2.legend();

ax2.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

#plt.tight_layout()



ax3 = fig3.add_subplot(3,2,5)


T3, E3, p3 = convergence1Lax(U_ref1,uExact1,g1,u0,f1,ft1,fx1)
ax3.loglog(T3,E3,'o-', label='p={:.2f}'.format(p3))
ax3.grid('on')
ax3.set_xlabel('t')
ax3.set_ylabel('error in time')
ax3.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax3.legend();


ax4 = fig3.add_subplot(3,2,6)


T3, E3, p3 = convergence1Lax(U_ref2,uExact2,g2,u0,f2,ft2,fx2)
ax4.loglog(T3,E3,'o-', label='p={:.2f}'.format(p3))
ax4.grid('on')
ax4.set_xlabel('t')

ax4.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax4.legend();
plt.savefig("LaxWplot.pdf")
